# Benchmarking [Mistral Moderation API](https://mistral.ai/fr/news/mistral-moderation)

The following tabular summarizes the categories that are evaluated by Mistral API, we put ✅ for the categories that enter our definition of `toxicity` and ❌ for those which are not considered as toxic.

| Catégorie                      | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|--------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Sexual** ✅                     | Matériel qui décrit, représente ou promeut explicitement des activités sexuelles, la nudité ou des services sexuels. Cela inclut le contenu pornographique, les descriptions graphiques d'actes sexuels et la sollicitation à des fins sexuelles. Le contenu éducatif ou médical sur la santé sexuelle présenté de manière non explicite et informative est généralement exempté.                                                        |
| **Hate and Discrimination** ✅   | Contenu exprimant des préjugés, de l'hostilité ou prônant la discrimination envers des individus ou groupes sur la base de caractéristiques protégées (race, ethnie, religion, genre, orientation sexuelle, handicap, etc.). Cela inclut les insultes, le langage déshumanisant, les appels à l'exclusion ou à la violence ciblée, ainsi que le harcèlement persistant fondé sur ces caractéristiques.                                  |
| **Violence and Threats** ✅      | Contenu décrivant, glorifiant, incitant ou menaçant de violence physique envers des individus ou groupes. Cela inclut les descriptions graphiques de blessures ou de décès, les menaces explicites de préjudice et les instructions pour commettre des actes violents. Cette catégorie couvre les menaces ciblées ainsi que la promotion ou la glorification générale de la violence.                                               |
| **Dangerous and Criminal Content** ✅ | Contenu promouvant ou fournissant des instructions pour des activités illégales ou des comportements extrêmement dangereux présentant un risque important de préjudice physique, de décès ou de conséquences juridiques. Cela inclut la fabrication d'armes ou d'explosifs, l'encouragement à des comportements à risque extrême, et la promotion de crimes non violents comme la fraude, le vol ou le trafic de drogue. |
| **Self-Harm** ✅                 | Contenu promouvant, instruisant, planifiant ou encourageant l'automutilation, le suicide, les troubles alimentaires ou d'autres comportements autodestructeurs. Cela inclut les méthodes détaillées, la glorification, les déclarations d'intention, les défis dangereux et le vocabulaire associé.                                                                                                                                |
| **Health** ❌                    | Contenu contenant ou cherchant à obtenir des conseils médicaux détaillés ou personnalisés.                                                                                                                                                                                                                                                                                                                                                                          |
| **Financial** ❌                 | Contenu contenant ou cherchant à obtenir des conseils financiers détaillés ou personnalisés.                                                                                                                                                                                                                                                                                                                                                                         |
| **Law** ❌                       | Contenu contenant ou cherchant à obtenir des conseils juridiques détaillés ou personnalisés.                                                                                                                                                                                                                                                                                                                                                                        |
| **PII** ❌                       | Contenu demandant, partageant ou tentant d'obtenir des informations personnelles identifiantes telles que noms complets, adresses, numéros de téléphone, numéros de sécurité sociale ou coordonnées bancaires.                                                                                                                                                                                                                                                        |

## Libraries

In [1]:
from mistralai import Mistral
from pathlib import Path
import os
import pandas as pd
from tqdm.rich import tqdm
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score
)
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
import warnings

tqdm.pandas(desc="Fetching moderation scores")
console = Console()
warnings.filterwarnings("ignore")

## Global variables

In [2]:
ROOT = Path("../..")
DATA_DIR = ROOT / "data"
BENCHMARK_PATH = DATA_DIR / "benchmark_jigsaw" / "benchmark_jigsaw.csv"
output_path = DATA_DIR / "benchmark_jigsaw" / "eng-mistral_moderation.csv"
API_KEY_PATH = DATA_DIR / "confidential" / "MISTRAL_API.txt"
console = Console()

In [3]:
os.environ["HTTP_PROXY"] = "socks5h://127.0.0.1:1080"
os.environ["HTTPS_PROXY"] = "socks5h://127.0.0.1:1080"

In [4]:
client = Mistral(api_key=API_KEY_PATH.read_text().strip())

## Load dataset

In [5]:
df = pd.read_csv(BENCHMARK_PATH, encoding="utf-8")
df = df.dropna(subset=["content", "label"])
df["label"] = df["label"].astype(int)

label_counts = df["label"].value_counts().reset_index()
label_counts.columns = ["label", "count"]
table = Table(title="Label Counts", show_lines=True)
table.add_column("Label", justify="center", style="cyan")
table.add_column("Count", justify="center", style="yellow")
table.add_row("Total Rows", str(len(df)))
for _, row in label_counts.iterrows():
    table.add_row(str(row["label"]), str(row["count"]))
console.print(table, justify="left")
df.sample(5, random_state=42)

     Label Counts                                                                                                  
┏━━━━━━━━━━━━┳━━━━━━━┓                                                                                             
┃   Label    ┃ Count ┃                                                                                             
┡━━━━━━━━━━━━╇━━━━━━━┩                                                                                             
│ Total Rows │  396  │                                                                                             
├────────────┼───────┤                                                                                             
│     0      │  198  │                                                                                             
├────────────┼───────┤                                                                                             
│     1      │  198  │                                                                                             
└────────────┴───────┘

,msg_id,original_text,content,label
78,ddb11abeca87bb3c,birth place \n\n YC was not born in Jonesboro...,lieu de naissance \n\nYC n'est pas né à Jones...,0
277,6e9826f166b7d7fe,Nazis like you deserve worse than death. \n\nI...,Des nazis comme toi méritent pire que la mort....,1
250,5bff63ef016af9be,You can rationalize your dishonesty however yo...,Tu peux rationaliser ton malhonnêteté comme tu...,1
55,e8ea81d9eac81735,Deletion\nThis has been tagged for Speedy Dele...,Suppression \nCeci a été marqué pour une supp...,0
391,bf5aae5b17fb0b77,because I am a huge faggot,parce que je suis un énorme pédé,1


## Define prediction function

In [6]:
def predict(text):
    response = client.classifiers.moderate(
        model = "mistral-moderation-latest",  
        inputs=[text],
    )
    return response

## Run prediction

In [7]:
df["toxicity_score"] = df["original_text"].progress_apply(predict)
df = df.dropna(subset=["toxicity_score"])

Output()

In [8]:
df['toxicity_score'][0].results[0].categories

{'sexual': False,
 'hate_and_discrimination': False,
 'violence_and_threats': False,
 'dangerous_and_criminal_content': False,
 'selfharm': False,
 'health': False,
 'financial': False,
 'law': False,
 'pii': False}

In [9]:
def is_toxic(score):
    categories = score.results[0].categories
    toxic_categories = ["sexual", "hate_and_discrimination", "violence_and_threats", "dangerous_and_criminal_content", "selfharm"]
    return any(categories.get(cat, False)for cat in toxic_categories)

In [10]:
df['prediction'] = df['toxicity_score'].apply(is_toxic).astype(int)

In [11]:
for i, row in df.sample(5, random_state=42).iterrows():
    content = Text(row['content'], style="bold")
    toxicity = f"[yellow]Toxicity Score:[/yellow] [bold]{row['prediction']}[/bold]"
    label = f"[cyan]Label:[/cyan] [bold]{row['label']}[/bold]"
    panel = Panel.fit(
        f"{content}\n\n{toxicity}\n{label}",
        title=f"Exemple {i+1}",
        border_style="magenta"
    )
    console.print(panel)

╭─────────────── Exemple 79 ───────────────╮
│ lieu de naissance                        │
│                                          │
│ YC n'est pas né à Jonesboro, en Géorgie. │
│                                          │
│ Toxicity Score: 0                        │
│ Label: 0                                 │
╰──────────────────────────────────────────╯

╭────────────────── Exemple 278 ──────────────────╮
│ Des nazis comme toi méritent pire que la mort.  │
│                                                 │
│ Je vais m'assurer que tu souffres horriblement. │
│                                                 │
│ Toxicity Score: 1                               │
│ Label: 1                                        │
╰─────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Exemple 251 ──────────────────────────────────────────────────╮
│ Tu peux rationaliser ton malhonnêteté comme tu veux, mais même Wikipédia a un article sur la mosquée de Ground  │
│ Zero, espèce de connard.                                                                                        │
│                                                                                                                 │
│ Toxicity Score: 1                                                                                               │
│ Label: 1                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Exemple 56 ─────────────────────────────────────────────╮
│ Suppression                                                                                         │
│ Ceci a été marqué pour une suppression rapide car c'est le même article précédemment supprimé sous. │
│                                                                                                     │
│ Toxicity Score: 0                                                                                   │
│ Label: 0                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────── Exemple 392 ───────────╮
│ parce que je suis un énorme pédé │
│                                  │
│ Toxicity Score: 1                │
│ Label: 1                         │
╰──────────────────────────────────╯

## Metrics & Report        

| Metric                     | Formula                                           | Interpretation                                                                                                       |
| -------------------------- | ------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **Precision**              | `TP / (TP + FP)`                                  | Of the samples predicted **toxic**, how many were **actually toxic**? <br>→ High precision = **low false positives** |
| **Recall** *(Sensitivity)* | `TP / (TP + FN)`                                  | Of the **actual toxic** samples, how many did we **correctly identify**? <br>→ High recall = **low false negatives** |
| **F1-score**               | `2 * (Precision * Recall) / (Precision + Recall)` | Harmonic mean of precision and recall. <br>→ Best when **balance** is needed                                         |
| **Accuracy**               | `(TP + TN) / (TP + TN + FP + FN)`                 | Fraction of all correct predictions (toxic and non-toxic). <br>→ Can be misleading on imbalanced data                |
| **ROC AUC**                | Area under the ROC Curve                          | Measures the **ranking ability** of the classifier. <br>→ Higher = better separation of toxic vs. non-toxic          |


In [12]:
y_true = df["label"]
y_pred = df["prediction"]

In [13]:
# Rapport de classification
report = classification_report(y_true, y_pred, digits=3, output_dict=True)
table = Table(title="Classification Report", show_lines=True)
table.add_column("Classe", style="cyan", justify="center")
table.add_column("Precision", justify="center")
table.add_column("Recall", justify="center")
table.add_column("F1-score", justify="center")
table.add_column("Support", justify="center")

for label, metrics in report.items():
    if label in ["accuracy", "macro avg", "weighted avg"]:
        continue
    table.add_row(
        str(label),
        f"{metrics['precision']:.3f}",
        f"{metrics['recall']:.3f}",
        f"{metrics['f1-score']:.3f}",
        f"{int(metrics['support'])}"
    )
# Ajout de l'accuracy
table.add_row(
    "[bold yellow]Accuracy[/bold yellow]",
    "-",
    "-",
    f"{report['accuracy']:.3f}",
    "-"
)
console.print(table)

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
cm_table = Table(title="Matrice de Confusion", show_lines=True)
cm_table.add_column(" ", style="bold")
cm_table.add_column("Prédit 0", justify="center")
cm_table.add_column("Prédit 1", justify="center")
cm_table.add_row("Réel 0", str(cm[0, 0]), str(cm[0, 1]))
cm_table.add_row("Réel 1", str(cm[1, 0]), str(cm[1, 1]))
console.print(cm_table)

# ROC AUC
try:
    auc = roc_auc_score(y_true, df["prediction"])
    console.print(Panel(f"[bold green]ROC AUC Score : {auc:.3f}[/bold green]", title="ROC AUC"))
except:
    console.print(Panel("[bold red]Impossible de calculer le ROC AUC.[/bold red]", title="ROC AUC"))

                Classification Report                 
┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃  Classe  ┃ Precision ┃ Recall ┃ F1-score ┃ Support ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│    0     │   0.874   │ 0.944  │  0.908   │   198   │
├──────────┼───────────┼────────┼──────────┼─────────┤
│    1     │   0.940   │ 0.864  │  0.900   │   198   │
├──────────┼───────────┼────────┼──────────┼─────────┤
│ Accuracy │     -     │   -    │  0.904   │    -    │
└──────────┴───────────┴────────┴──────────┴─────────┘

      Matrice de Confusion      
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃        ┃ Prédit 0 ┃ Prédit 1 ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ Réel 0 │   187    │    11    │
├────────┼──────────┼──────────┤
│ Réel 1 │    27    │   171    │
└────────┴──────────┴──────────┘

╭──────────────────────────────────────────────────── ROC AUC ────────────────────────────────────────────────────╮
│ ROC AUC Score : 0.904                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Save the predictions

In [14]:
df

,msg_id,original_text,content,label,toxicity_score,prediction
0,e6865c831da40459,Combining 'Official requests for clemency' sec...,Combiner la section 'Demandes officielles de c...,0,id='fce8f1db4b34465fbfeb0f177a6f780d' model='m...,0
1,cff2b456e5059b47,Web \n\nIs this her own website? 113.22.84.65,Web\n\nEst-ce son propre site web ? 113.22.84.65,0,id='764eadaaff28492da0eadb61bd1eef6c' model='m...,0
2,880d3d608f37bee3,"RFCU \n\nFYI, I have opened Wikipedia:Requests...","RFCU \n\nPour info, j'ai ouvert Wikipedia:Requ...",0,id='82eaf9743bbb4f22bc7aae986124576f' model='m...,0
3,dd29d52a4320a2e4,Monitor for cleanup: \nAs of the date signed b...,Surveillance pour nettoyage : \nÀ la date sig...,0,id='f2ead21d55b042cbaee21c28cfb1b9d7' model='m...,0
4,d70e773c975839e1,I concur and have done so.,Je suis d'accord et je l'ai fait.,0,id='5468bed552c440db8c87111c6d9d09a3' model='m...,0
...,...,...,...,...,...,...
391,bf5aae5b17fb0b77,because I am a huge faggot,parce que je suis un énorme pédé,1,id='fae8b2c8e9ab4cef9de2a775d97e6946' model='m...,1
392,8947f2c12aea0a32,"If no-one beats me to it, I'll knock something...","Si personne ne me devance, je vais bricoler qu...",1,id='94590f806875498cbcbd942f839c4daf' model='m...,0
393,ccf05c5d4b791836,"ur a faggot, fuck off and get a real job and s...","T'es un pédé, casse-toi et trouve un vrai boul...",1,id='c961ae115d5e4230a7620e9c93ed1cf8' model='m...,1
394,ae7a53526025c0e2,"I SAID SHUT UP, YOU STUPID, STUPID...","J'AI DIT FERME TA GUEULE, ESPÈCE DE BÊTE, BÊTE...",1,id='8d3a8f2c43d04d3ca4630dd7c3e7a613' model='m...,1


In [15]:
output_path

PosixPath('../../data/benchmark_jigsaw/eng-mistral_moderation.csv')

In [16]:
df.to_csv(output_path, index=False, encoding="utf-8")